### Bibliotecas

In [4]:
import pandas as pd
import re
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import metrics as mt
import seaborn as sns
import matplotlib.pyplot as plt
import os



#### Ler Dataset no formato csv

In [ ]:
df_raw = pd.read_csv('..\dataset\database_credit.csv') 
 
df_raw.head()

#### Dimensão dos dados 

In [ ]:
print("\nDimensões Credit Dataset:\n{0}\n".format(df_raw.shape))
print("\nCampos de Credit Dataset:\n{0}\n".format(list(df_raw.keys())))
print("\nTipos dos dados:\n{0}\n".format(df_raw.dtypes))

### Valores Ausentes 

In [ ]:
# Calcular a porcentagem de  em cada coluna
missing_perc = df_raw.isna().mean() * 100
missing_perc


####  Seleção de Features/Características

In [ ]:
# copy dataset
df = df_raw.copy()

features = ['idade', 'divida_atual', 'renda_anual', 'valor_em_investimentos',
            'taxa_utilizacao_credito', 'num_emprestimos', 'num_contas_bancarias', 'num_cartoes_credito',
            'dias_atraso_dt_venc', 'num_pgtos_atrasados', 'num_consultas_credito', 'taxa_juros']

#### Selecionando Variável alvo

In [ ]:
# Especificando a variável alvo que o modelo tentará prever. Neste caso, é o "saldo_atual".
label = ['saldo_atual']

##### Dados de Treino e Teste

In [ ]:
"""Aqui, estamos dividindo o conjunto de dados em duas partes: x_train,
  que contém as características que serão usadas para treinar o modelo,
  e y_train, que contém os valores da variável alvo correspondentes."""

# Dados de treinamento e teste
x_train = df.loc[:, features]
y_train = df.loc[:, label]

In [ ]:
"""
Agora criando o modelo de regressão linear (LinearRegression())
e, em seguida, treinando esse modelo com os dados de treinamento usando o método fit().
O modelo tentará aprender uma relação linear entre as características e a variável alvo.
"""
# Treinamento do algoritmo
lr_model = LinearRegression()
lr_model.fit( x_train, y_train )

In [ ]:
"""
Agora que o modelo está treinado, estamos usando-o para fazer previsões sobre a variável alvo -
com base nas características do conjunto de treinamento.
As previsões são armazenadas na variável chamada y_pred.
"""

# Previsão do algoritmo
y_pred = lr_model.predict( x_train )

#### Visualizando os resultados previstos

In [ ]:
"""
Aqui, estamos criando um novo DataFrame chamado df1,
que contém as colunas 'id_cliente', 'saldo_atual' (real) e 'predicted' (previsão).
O método head() é utilizado para visualizar as primeiras linhas do DataFrame resultante,
mostrando como as previsões se comparam aos valores reais.
"""


# Resultado final
df1 = df.loc[:, ['id_cliente', 'saldo_atual']]
df1['predicted'] = y_pred

df1.head()

#### Função criada para retornar as métricas


In [ ]:
def regression_metrics(y_train, y_pred, outlier):
    r2 = mt.r2_score(y_train, y_pred)
    mse = mt.mean_squared_error(y_train, y_pred)
    rmse = np.sqrt(mse)

    metrics = pd.DataFrame({'outliers': [outlier], 'r2': [r2], 'mse': [mse], 'rmse': [rmse]})

    return metrics

In [ ]:
regression_metrics(y_train, y_pred, 0)

### Avaliação dos resultados e métricas retornadas pela função:

#### 1- Outliers:

Valor: 0
Isso indica que o parâmetro outlier foi definido como 0. No contexto do código, é uma indicação de que não há outliers considerados no cálculo das métricas.

#### 2- Coeficiente de Determinação (R²):

Valor: 0.17
O coeficiente de determinação (R²) varia de 0 a 1. Neste caso, o valor de 0.17 mostra que aproximadamente 17% da variabilidade na variável alvo (saldo_atual) é explicada pelas características fornecidas no modelo. Um R² de 0.17 indica que o modelo tem um ajuste relativamente modesto aos dados.

#### 3- Erro Médio Quadrático (MSE):

Valor: 39370.27
O MSE é uma medida da média dos quadrados dos erros entre as previsões e os valores reais. Neste caso, um MSE de 39370.27 indica que, em média, os quadrados dos erros são bastante elevados. Isso significa que o modelo pode estar cometendo grandes erros nas previsões.

#### 4- Raiz Quadrada do Erro Médio Quadrático (RMSE):

Valor: 198.42
O RMSE é a raiz quadrada do MSE e fornece uma métrica na mesma escala que a variável alvo. Neste caso, um RMSE de 198.42 indica que o modelo tem, em média, uma discrepância de cerca de 198.42 unidades (na mesma escala da variável alvo) em suas previsões em relação aos valores reais.


#### <<Interpretação:>>
Os resultados sugerem que o modelo de regressão linear pode não estar fornecendo um ajuste muito preciso aos dados, pois o R² é relativamente baixo, e o MSE e RMSE são altos. Isso pode indicar que o modelo pode precisar ser ajustado ou que outros modelos mais complexos podem ser considerados para melhorar a performance de previsão.

## Para fixar um pouco mais sobre o estudo, aprendi e "sugiro" sempre testar novas possibilidades no mesmo conjunto de dados.  Neste contexto modifiquei 8% das linhas de algumas colunas, adicionei outliers e refiz as métricas de R2, MSE e RMSE.

#### Função criada para gerar outliers

In [ ]:
def random_outliers(df, percentual, features, peso, seed):

  """
  features: Colunas na qual os outliers serão adidionados
  percentual: Percentual de linhas do dataframe que serão afetadas pelos outliers
  peso: Fator de multiplicação para os outliers
  seed: Garante que toda vez que a função for executada com os mesmos parametros, gere os mesmos registros.

  """
  np.random.seed(seed)
  random_index = np.random.randint(1,df.shape[0], int(df.shape[0]*percentual))
  df.loc[random_index, features] = df.loc[random_index, features] *peso

  return df

In [ ]:
# Criando uma cópia do dataframe de treinamento
X_train2 = x_train.copy()

In [ ]:

#Criando os parametros que desejo para aplicar aos Outliers

""" 
percentual: Representa o percentual de linhas que serão modificadas. No caso, 8%.
features: Lista das colunas (características) que serão modificadas.
peso: Um fator multiplicativo que será usado para adicionar outliers. No caso, é 10.
"""

percentual = 0.08
features = ['idade', 'divida_atual', 'renda_anual']
peso = 10
seed = 25

In [ ]:
# Aplicando a função
X_train2 = random_outliers(X_train2, percentual, features, peso, seed)

In [ ]:
"""
Esta linha de código exibe as estatísticas descritivas das colunas especificadas (features) após a modificação.
As estatísticas descritivas incluem informações como média, desvio padrão, mínimo, máximo, etc.
O método round é utilizado para arredondar os valores e torná-los mais legíveis.
Portanto, após essa modificação, você terá um conjunto de dados (X_train2) com algumas linhas modificadas em relação ao conjunto original (x_train).
Essa abordagem permite avaliar como a introdução de outliers afeta as métricas de desempenho do seu modelo.

"""

np.round(X_train2[features].describe().T)

In [ ]:
"""
Utilizando o seu modelo de regressão linear (lr_model), estamos agora previsões baseadas no conjunto de dados-
modificado (X_train3). Isso resultará em um novo conjunto de previsões (y_pred2), que agora reflete o impacto dos outliers.
"""

y_pred2 = lr_model.predict(X_train2)

In [ ]:
# Avaliação das Métricas de Desempenho:
regression_metrics(y_train,y_pred2, percentual)

## Bora fazer mais alguns testes!

### Novamente vamos criar novos outliers, mais agora com as seguinte percentual de quantidades: 2%, 5%, 10%, 20% e 30%.

##### Função criada para projetar e avaliar o impacto de diferentes niveis de Outliers

In [ ]:
def metricas_outliers(df, percentuais, features, peso, seed):
    """
    A função metricas_outliers vai retornar um DataFrame com as métricas de desempenho do modelo (R², MSE, RMSE) para cada nível de percentual de outliers.
    Isso permite analisar como a inclusão de diferentes proporções de outliers o quanto afeta a precisão e a confiabilidade do modelo.

    """
    result = pd.DataFrame({'outliers': [], 'r2': [], 'mse': [], 'rmse': []})
    for i in percentuais:
        data = df.copy()
        data = random_outliers( data, i, features, peso, seed)
        y_pred = lr_model.predict( data )

        metrics = regression_metrics(y_train, y_pred, i)

        result = pd.concat([result, metrics], axis=0)

    return np.round(result,2)

In [ ]:
data = x_train.copy()
percentuais = [0.02, 0.05, 0.1, 0.2, 0.3]
features = ['idade', 'divida_atual', 'renda_anual']
peso = 10
seed = 25

In [ ]:
metricas = metricas_outliers( data, percentuais, features, peso, seed)
metricas